<a href="https://colab.research.google.com/github/nakkata/dev_tool/blob/main/Public_Custom_Knowledge_ChatGPT_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChainを使ってChatGPTに独自データに関する質問に回答してもらう

このサンプルでは、インターネット上からダウンロードしたPDFファイルに関する質問に対しての回答をしてもらいます。
PDFファイルはインターネットからダウンロードします。
URLを変更するなどすることで、独自のPDFファイルを代わりに読み込ませることも可能です。


以下に処理の概要を示します。

0.   OpenAPIキーの設定と、関連ライブラリのインストールとインポート
1.   PDFをロードし、LangChainを使ってチャンクに分割する
2.   テキスト情報を数値化（Embedding)し、保存する
3.   データ取得関数を作成する。
4.   チャットbotの作成

こちらの記事は、Liam Ottleyさんの記事を参考にして独自の情報を追加しています。  
動画は以下のYoutubeからご参照ください。
  [YouTube](https://youtube.com/@LiamOttley)





# 0. OpenAPIキーの設定と、関連ライブラリのインストール
利用するライブラリをpipを使ってインストールします。

In [ ]:
# RUN THIS CELL FIRST!//
!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu requests beautifulsoup4

In [ ]:
import os
import pandas as pd
import requests
import textract
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


以下にOpenAIから取得したキーを設定します。キーは以下のURLから取得することができます。  
（アカウントの作成が必要です）  
https://platform.openai.com/account/api-keys



In [ ]:
# ここにOpenAIから取得したキーを設定します。
os.environ["OPENAI_API_KEY"] = 'sk-sssss'

インターネット上からPDFファイル（日本語で書かれたマニュアル）をダウンロードして、ローカルファイルとして保存します。

In [ ]:
# インターネット上からマニュアルファイルをダウンロード（BuffaloTerastation)
url = 'https://manual.buffalo.jp/buf-doc/35021178-39.pdf'
response = requests.get(url)
# Ensure that the request was successful
if response.status_code == 200:
    # Save the content of the response as a PDF file
    with open('sample_document1.pdf', 'wb') as file:
        file.write(response.content)
else:
    print("Error: Unable to download the PDF file. Status code:", response.status_code)


In [ ]:
# インターネット上からマニュアルファイルをダウンロード（サイボウズOfficeユーザマニュアル）
# url = 'https://jp.cybozu.help/o/pdf/manual_o_user.pdf'
# response = requests.get(url)
# # Ensure that the request was successful
# if response.status_code == 200:
#     # Save the content of the response as a PDF file
#     with open('sample_document2.pdf', 'wb') as file:
#         file.write(response.content)
# else:
#     print("Error: Unable to download the PDF file. Status code:", response.status_code)


# 1. データをダウンロードし、LangChainを使ってチャンクに分割する
ここでは、自社の情報と仮定して、2つのPDFファイルを読み込みます。必要に応じて読み込むファイルを自社のマニュアルなどに置き換えてみてください。




### ドキュメント1のロード＆チャンク分割

In [ ]:
# ページごとに分割。この方法だと、メタデータの情報が取得できるので、マニュアルのページ数などを表示することも可能となるが、
# トークンサイズが大きくなりがち。
# また、PDFの様なページ分割されている情報がソースとなっている必要がある
# Simple method - Split by pages
# https://manual.buffalo.jp/buf-doc/35021178-39.pdf
#loader = PyPDFLoader("/content/sample_data/buffalo_manual.pdf")
loader = PyPDFLoader("sample_document1.pdf")
pages = loader.load_and_split()
print(pages[3])

# SKIP TO STEP 2 IF YOU'RE USING THIS METHOD
chunks = pages

page_content='外付けUSBドライブを増設する  ............................................... 75\nドライブの取り外し処理をする  ............................................... 76\nファンクションボタンを使って取り外す  .......................................... 76\n設定画面を使って取り外す  ................................................................ 77\nドライブをチェックする  .......................................................... 78\nSSDのTrimを設定する （TS5210DFシリーズのみ）  .................. 79\nSSDのデータ保護機能について （TS5210DFシリーズのみ）  ....81\n制限される機能 .................................................................................. 81\nSSDの総書き込み容量の上限を設定する （TS5210DFシリーズの\nみ） ............................................................................................ 83\nS.M.A.R.T.で内蔵ドライブをチェックする  .............................. 84\nS.M.A.R.T.を表示する  ....................................................................... 84\nドライブ状態の判定のしかた  ............................................................ 86\nドライブをフォーマットする  ..............................................

### ドキュメント2のロード＆チャンク分割



In [ ]:
# # 上級者向けの方法　チャンクに分割。これだとメタデータの情報が取れない

# # Step 1: Convert PDF to text
# doc = textract.process("sample_document2.pdf")
# # Step 2: Save to .txt and reopen (helps prevent issues)
# with open('attention_is_all_you_need.txt', 'w') as f:
#     f.write(doc.decode('utf-8'))

# with open('attention_is_all_you_need.txt', 'r') as f:
#     text = f.read()

# # Step 3: Create function to count tokens
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# def count_tokens(text: str) -> int:
#     return len(tokenizer.encode(text))

# # Step 4: Split text into chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size = 512,
#     chunk_overlap  = 24,
#     length_function = count_tokens,
# )

# chunks2 = text_splitter.create_documents([text])

## ドキュメント3のロード＆チャンク分割

In [ ]:
# chank3つ目。3つ目は、インターネット上のドキュメントから情報を取得する
url = "https://mui.com/material-ui/getting-started/overview/"
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to fetch the webpage")

# インターネット上のサイトから抽出した情報をDBに入れる
# Step 2: Save to .txt and reopen (helps prevent issues)
with open('internet_info1.txt', 'w') as f:
    f.write(html_content)

with open('internet_info1.txt', 'r') as f:
    text = f.read()

# Step 3: Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# Step 4: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    # chunk_size = 512,
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks3 = text_splitter.create_documents([text])

Token indices sequence length is longer than the specified maximum sequence length for this model (40701 > 1024). Running this sequence through the model will result in indexing errors


# 2. 読み込んだデータvector storeに保存する
ここでは、これまでに読み込んだ3つの情報をVector storeに保存します。。
Vector storeはいくつも種類がありますが、今回はFacebook製のFAISSを利用します。  


> 利用可能なVectorStoreはこちらを参照してください。
https://python.langchain.com/en/latest/modules/indexes/vectorstores.html#




In [ ]:
# Get embedding model
embeddings = OpenAIEmbeddings()

#  vector databaseの作成
db = FAISS.from_documents(chunks+chunks3, embeddings)

# 3. データ取得関数のセットアップ

次に、先ほど作ったFAISSのvector storeにqueryを投げてどの様なデータが返ってくるかを確認します。

In [ ]:
query = "ランプが点滅しているが、これは何が原因か？"
# FAISSに対して検索。検索は文字一致ではなく意味一致で検索する(Vector, Embbeding)
docs = db.similarity_search(query)
docs # ここで関係のありそうなデータが返ってきていることを確認できる

[Document(page_content='電源ランプが点滅したまま点灯に変わらない\n1 電源ボタンを約3秒間長押しして、 本製品の電源をOFFにします。\n2 ファンクションボタンを押したまま、 電源ボタンを押します。 電源ボタンを放した後も、 ファンクション\nボタンは約10秒間押し続けてください。\n3 電源ランプが点滅から点灯に変わったら、 NAS Navigator2で本製品を検索し、 設定画面を開きます。\n4 以下の画面が表示されます。 「セットアップ」 の下のプルダウンメニューで 「修復」 が選択されていること\nを確認して、 ［セットアップ開始］ をクリックします。\nメモ: 「セットアップ」 には 「修復」 以外にも以下のオプションが表示される場合があります。 それぞれの\nオプションの動作は以下のとおりです。\n• FW Ver維持 ： 本製品内蔵のドライブに保存されているファームウェアをそのまま使用します。\n• 修復： 本製品のNANDフラッシュに保存されているファームウェアを使って本製品を修復します。\n• フォーマット ： ドライブをフォーマットします。\n5 「通信の確認」 画面が表示されます。 表示されている数字を正確に入力し、 ［OK］ をクリックします。\n6 ファームウェアが再インストールされます。 完了したら、 ［OK］ をクリックします。\n7 本製品は自動的にシャットダウンします。 電源ランプが消灯したら、 電源ボタンを押して起動します。\n以上で、 電源ランプが点滅したままの状態から復旧する手順は完了です。 本製品が正常に起動しているか確認\nしてください。\nメモ: \n• 上記手順で現象が改善されないときは、 もう一度初めから操作してください。 そのとき、 手順4の画面で 「最\n後に正常起動したときの設定を復元する」 を選択して、 ［セットアップ開始］ をクリックしてください。 前回\n正常に起動したときの設定が復元された状態で、 本製品が起動します。\n323\n第１２章\u3000トラブルシューティングとメンテナンス', metadata={'source': 'sample_document1.pdf', 'page': 323}),
 Document(page_content='ステータスランプが

## ユーザからのクエリを使って関連するデータを取得できる様にQAチェインを作成する

ここで、これまでにロードしたドキュメントに書かれた情報に関する質問を投げてみて、期待する結果が返ってくるかどうかを確認します。


>  以下コード内のtemperatureを変更することにより、情報の精度を上げることができます。(0-2までの値で指定）0にした場合、質問内容の回答がはっきりわからない場合はI don't knowと言われます。。



In [ ]:
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.1,max_tokens=1000), chain_type="stuff")
# p305に記載
#query = "ドライブのランプが赤色に点滅しているが、これは何が原因か？"
# p134に記載
#query = "どの様な時にメイン機が異常だと判断をしますか？"
query = "バックアップにはどの様な方法がありますか？またその手順についておしえてください"
docs = db.similarity_search(query)
# langchainを使って検索
chain.run(input_documents=docs, question=query)

' バックアップには、「公開プロトコル」の「バックアップ」を使用する方法があります。手順としては、まず「バックアップディスクを追加」をクリックし、「公開プロトコル」の「バックアップ」にチェックマークを付けます。次に、バックアップ先の共有フォルダーに設定したバックアップアクセスキーを入力し、「追加」をクリックします。最後に、「バックアップアクセスキー」欄に任意の文字を入力し、「OK」をクリックします。'

# 4. チャットbotを作成する（Option)
ここでは、これまでに読み込んだドキュメントを使って簡易のチャットボットを作成します。
ここを実行するとダイアログが表示されますので、前に読み込ませたドキュメントに関する質問をして見てください。

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# vextordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

In [ ]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Welcome to the Transformers chatbot! Type 'exit' to stop.


Text(value='', placeholder='Please enter your question:')